![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/KEYPHRASE_EXTRACTION.ipynb)

# **Extract keyphrases from documents**

You can look at the example outputs stored at the bottom of the notebook to see what the model can do, or enter your own inputs to transform in the "Inputs" section. Find more about this keyphrase extraction model in another notebook [here](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/9.Keyword_Extraction_YAKE.ipynb).

## 1. Colab setup

Install dependencies

In [5]:
# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp

/bin/sh: apt-get: command not found
/bin/sh: apt-get: command not found
java version "1.8.0_271"
Java(TM) SE Runtime Environment (build 1.8.0_271-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.271-b09, mixed mode)


In [1]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
#create or get Spark Session
spark = sparknlp.start()
sparknlp.version()
spark.version
#download, load, and annotate a text by pre-trained pipeline
pipeline = PretrainedPipeline('recognize_entities_dl', 'en')
result = pipeline.annotate('Harry Potter is a great movie')

recognize_entities_dl download started this may take some time.
Approx size to download 159 MB
[OK!]


Import dependencies

In [3]:
import json
import os
import pandas as pd
import numpy as np

os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

# Import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

# Import SparkNLP
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

# Start Spark session
spark = sparknlp.start()

## 2. Inputs

Enter inputs as strings in this list. Later cells of the notebook will extract keyphrases from whatever inputs are entered here.

In [75]:
input_list = [
    """Extracting keywords from texts has become a challenge for individuals and organizations as the information grows in complexity and size. The need to automate this task so that text can be processed in a timely and adequate manner has led to the emergence of automatic keyword extraction tools. Yake is a novel feature-based system for multi-lingual keyword extraction, which supports texts of different sizes, domain or languages. Unlike other approaches, Yake does not rely on dictionaries nor thesauri, neither is trained against any corpora. Instead, it follows an unsupervised approach which builds upon features extracted from the text, making it thus applicable to documents written in different languages without the need for further knowledge. This can be beneficial for a large number of tasks and a plethora of situations where access to training corpora is either limited or restricted.""",
    """Iodine deficiency is a lack of the trace element iodine, an essential nutrient in the diet. It may result in metabolic problems such as goiter, sometimes as an endemic goiter as well as cretinism due to untreated congenital hypothyroidism, which results in developmental delays and other health problems. Iodine deficiency is an important global health issue, especially for fertile and pregnant women. It is also a preventable cause of intellectual disability.

Iodine is an essential dietary mineral for neurodevelopment among offsprings and toddlers. The thyroid hormones thyroxine and triiodothyronine contain iodine. In areas where there is little iodine in the diet, typically remote inland areas where no marine foods are eaten, iodine deficiency is common. It is also common in mountainous regions of the world where food is grown in iodine-poor soil.

Prevention includes adding small amounts of iodine to table salt, a product known as iodized salt. Iodine compounds have also been added to other foodstuffs, such as flour, water and milk, in areas of deficiency. Seafood is also a well known source of iodine.""",
    """The Prague Quadrennial of Performance Design and Space was established in 1967 to bring the best of design for performance, scenography, and theatre architecture to the front line of cultural activities to be experienced by professional and emerging artists as well as the general public. The quadrennial exhibitions, festivals, and educational programs act as a global catalyst of creative progress by encouraging experimentation, networking, innovation, and future collaborations. PQ aims to honor, empower and celebrate the work of designers, artists and architects while inspiring and educating audiences, who are the most essential element of any live performance. The Prague Quadrennial strives to present performance design as an art form concerned with creation of active performance environments, that are far beyond merely decorative or beautiful, but emotionally charged, where design can become a quest, a question, an argument, a threat, a resolution, an agent of change, or a provocation. Performance design is a collaborative field where designers mix, fuse and blur the lines between multiple artistic disciplines to search for new approaches and new visions.

The Prague Quadrennial organizes an expansive program of international projects and activities between the main quadrennial events – performances, exhibitions, symposia, workshops, residencies, and educational initiatives serve as an international platform for exploring the practice, theory and education of contemporary performance design in the most encompassing terms.""",
    """Author Nathan Wiseman-Trowse explained that the "approach to the sheer physicality of sound" integral to dream pop was "arguably pioneered in popular music by figures such as Phil Spector and Brian Wilson". The music of the Velvet Underground in the 1960s and 1970s, which experimented with repetition, tone, and texture over conventional song structure, was also an important touchstone in the genre's development George Harrison's 1970 album All Things Must Pass, with its Spector-produced Wall of Sound and fluid arrangements, led music journalist John Bergstrom to credit it as a progenitor of the genre.

Reynolds described dream pop bands as "a wave of hazy neo-psychedelic groups", noting the influence of the "ethereal soundscapes" of bands such as Cocteau Twins. Rolling Stone's Kory Grow described "modern dream pop" as originating with the early 1980s work of Cocteau Twins and their contemporaries, while PopMatters' AJ Ramirez noted an evolutionary line from gothic rock to dream pop. Grow considered Julee Cruise's 1989 album Floating into the Night, written and produced by David Lynch and Angelo Badalamenti, as a significant development of the dream pop sound which "gave the genre its synthy sheen." The influence of Cocteau Twins extended to the expansion of the genre's influence into Cantopop and Mandopop through the music of Faye Wong, who covered multiple Cocteau Twins songs, including tracks featured in Chungking Express, in which she also acted. Cocteau Twins would go on to collaborate with Wong on original songs of hers, and Wong contributed vocals to a limited release of a late Cocteau Twins single.

In the early 1990s, some dream pop acts influenced by My Bloody Valentine, such as Seefeel, were drawn to techno and began utilizing elements such as samples and sequenced rhythms. Ambient pop music was described by AllMusic as "essentially an extension of the dream pop that emerged in the wake of the shoegazer movement", distinct for its incorporation of electronic textures.

Much of the music associated with the 2009-coined term "chillwave" could be considered dream pop. In the opinion of Grantland's David Schilling, when "chillwave" was popularized, the discussion that followed among music journalists and bloggers revealed that labels such as "shoegaze" and "dream pop" were ultimately "arbitrary and meaningless".""",
    """North Ingria was located in the Karelian Isthmus, between Finland and Soviet Russia. It was established 23 January 1919. The republic was first served by a post office at the Rautu railway station on the Finnish side of the border. As the access across the border was mainly restricted, the North Ingrian postal service was finally launched in the early 1920. The man behind the idea was the lieutenant colonel Georg Elfvengren, head of the governing council of North Ingria. He was also known as an enthusiastic stamp collector. The post office was opened at the capital village of Kirjasalo.

The first series of North Ingrian stamps were issued in 21 March 1920. They were based on the 1917 Finnish "Model Saarinen" series, a stamp designed by the Finnish architect Eliel Saarinen. The first series were soon sold to collectors, as the postage stamps became the major financial source of the North Ingrian government. The second series was designed for the North Ingrian postal service and issued 2 August 1920. The value of both series was in Finnish marks and similar to the postal fees of Finland. The number of letters sent from North Ingria was about 50 per day, most of them were carried to Finland. They were mainly sent by the personnel of the Finnish occupying forces. Large number of letters were also sent in pure philatelic purposes.

With the Treaty of Tartu, the area was re-integrated into Soviet Russia and the use of the North Ingrian postage stamps ended in 4 December 1920. Stamps were still sold in Finland in 1921 with an overprinting "Inkerin hyväksi" (For the Ingria), but they were no longer valid. Funds of the sale went for the North Ingrian refugees.""",
    """Учитесь учиться! Мы вступаем в век, в котором образование, знания, профессиональные навыки будут определяющую роль в судьбе человека. Без знаний, кстати сказать, всё усложняющихся, просто нельзя будет работать, приносить пользу. Ибо физический труд возьмут на себя машины, роботы. Даже вычисления будут делаться компьютерами, так же как чертежи, расчёты, отчёты, планирование и т.д. Человек будет вносить новые идеи, думать над тем, над чем не сможет думать машина. А для этого всё больше нужна будет общая интеллигентность человека, его способность создавать новое и, конечно, нравственная ответственность, которую никак не сможет нести машина. Этика, простая в предшествующие века, бесконечно усложнится в век науки. Это ясно. Значит, на человека ляжет тяжелейшая и сложнейшая задача быть человеком не просто, а человеком науки, человеком, нравственно отвечающим за всё, что происходит в век машин и роботов. Общее образование может создать человека будущего, человека творческого, созидателя всего нового и нравственно за всё, что будет создаваться. Учение – вот что сейчас нужно молодому человеку с самого малого возраста. Учиться нужно всегда. До конца жизни не только учили, но и учились все крупнейшие учёные. Перестанешь учиться - не сможешь и учить. Ибо знания всё растут и усложняются. Нужно при этом помнить, что самое благоприятное время для учения – молодость. Именно в молодости, в детстве, в отрочестве, в юности ум человека наиболее восприимчив. Восприимчив к изучению языков (что крайне важно), к математике, к усвоению просто знаний и развитию эстетическому, стоящему рядом с развитием нравственным и отчасти его стимулирующим. Умейте не терять времени на пустяки, на «отдых», который иногда утомляет больше, чем самая тяжёлая работа, не заполняйте свой светлый разум мутными потоками глупой и бесцельной «информации». Берегите себя для учения, для приобретения знаний и навыков, которые только в молодости вы освоите легко быстро. И вот тут я слышу тяжкий вздох молодого человека: какую же скучную жизнь вы предлагаете нашей молодёжи! Только учиться. А где же отдых, развлечения? Что же нам, и не радоваться? Да нет же. Приобретение навыков и знаний – это тот же спорт. Учение тяжело, когда мы не умеем найти в нём радость. Надо уметь учиться и формы отдыха и развлечений выбирать умные, способные также чему-то научить, развить в нас какие-то способности, которые понадобятся в жизни. А если не нравится учиться? Быть того не может. Значит, вы просто не открыли той радости, которую приносит ребёнку, юноше, девушке приобретение знаний и навыков. Посмотрите на маленького ребёнка – с каким удовольствием он начинает учиться ходить, говорить, копаться в различных механизмах (у мальчиков), нянчить куклы (у девочек). Постарайтесь продолжить эту радость освоения нового. Это во многом зависит именно от вас самих. Не зарекайтесь: не люблю учиться! А вы попробуйте любить все предметы, какие проходите в школе. Если другим людям они нравятся, то почему вам они могут не понравиться! Читайте стоящие книги, а не просто чтиво. Изучайте историю и литературу. И то и другое должен хорошо знать интеллигентный человек. Именно они дают человеку нравственный и эстетический кругозор, делающий окружающий мир большим, интересным, излучающим опыт и радость. Если вам что-то не нравится в каком-то предмете – напрягитесь и постарайтесь найти в нём источник радости – радости приобретения нового. Учитесь любить учиться!"""
]

# Change these to wherever you want your inputs and outputs to go
INPUT_FILE_PATH = "inputs"
OUTPUT_FILE_PATH = "outputs"

#### Пытался загружать текст из файла, 

но когда запускаю df = spark.createDataFrame(pd.DataFrame({'text': input_list})), выдает ошибку

In [28]:
with open("inputs/Lihachev_learn.txt", "r") as file:
    input_list = file.read()
input_list

'\ufeffУчитесь учиться!\n\n  Мы вступаем в век, в котором образование, знания, профессиональные навыки будут определяющую роль в судьбе человека. Без знаний, кстати сказать, всё усложняющихся, просто нельзя будет работать, приносить пользу. Ибо физический труд возьмут на себя машины, роботы. Даже вычисления будут делаться компьютерами, так же как чертежи, расчёты, отчёты, планирование и т.д. Человек будет вносить новые идеи, думать над тем, над чем не сможет думать машина. А для этого всё больше нужна будет общая интеллигентность человека, его способность создавать новое и, конечно, нравственная ответственность, которую никак не сможет нести машина. Этика, простая в предшествующие века, бесконечно усложнится в век науки. Это ясно.\n  \n  Значит, на человека ляжет тяжелейшая и сложнейшая задача быть человеком не просто, а человеком науки, человеком, нравственно отвечающим за всё, что происходит в век машин и роботов. Общее образование может создать человека будущего, человека творческог

Write the example inputs to the input folder.

In [49]:
! mkdir -p $INPUT_FILE_PATH

for i, text in enumerate(input_list):
    open(f'{INPUT_FILE_PATH}/Example{i + 1}.txt', 'w') \
        .write(text[:min(len(text) - 10, 100)] + '... \n' + text)

## 3. Pipeline creation

Create the NLP pipeline.

In [99]:
# Transforms the raw text into a document readable by the later stages of the
# pipeline
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

# Separates the document into sentences
sentence_detector = SentenceDetector() \
    .setInputCols(['document']) \
    .setOutputCol('sentences')# \
    #.setDetectLists(True)

# Separates sentences into individial tokens (words)
tokenizer = Tokenizer() \
    .setInputCols(['sentences']) \
    .setOutputCol('tokens') \
    .setContextChars(['(', ')', '?', '!', '.', ','])

# Модель извлечения ключевых фраз. Измените MinNGrams и MaxNGrams, 
# чтобы установить минимальную и максимальную длину возможных ключевых фраз, и 
# измените NKeywords, чтобы установить количество потенциальных ключевых фраз, идентифицированных в каждом документе.
keywords = YakeModel() \
    .setInputCols('tokens') \
    .setOutputCol('keywords') \
    .setMinNGrams(2) \
    .setMaxNGrams(15) \
    .setNKeywords(1000) \
    .setStopWords(StopWordsCleaner().getStopWords())

# Assemble all of these stages into a pipeline, then fit the pipeline on an
# empty data frame so it can be used to transform new inputs.
pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    keywords
])
empty_df = spark.createDataFrame([[""]]).toDF('text')
pipeline_model = pipeline.fit(empty_df)

# LightPipeline работает быстрее, чем Pipeline для небольших наборов данных
light_pipeline = LightPipeline(pipeline_model)

## 4. Output creation

Служебные функции для создания более полезных наборов ключевых фраз из необработанного фрейма данных, создаваемого моделью.

In [58]:
def adjusted_score(row, pow=2.5):
    """Эта функция настраивает оценки потенциальных ключевых фраз, чтобы дать лучшие
    оценки фразам с большим количеством слов (которые, естественно, будут иметь худшие оценки
    из-за природы модели). Вы можете изменить показатель, чтобы более или менее вознаграждать более длинные фразы. 
    Более высокие показатели вознаграждают более длинные фразы."""
    return ((row.result.count(' ') + 1) ** pow /
            (float(row.metadata['score']) + 0.1))

def get_top_ranges(phrases, input_text):
    """Комбинируйте фразы, которые накладываются друг на друга"""
    starts = sorted([row['begin'] for row in phrases])
    ends = sorted([row['end'] for row in phrases])

    ranges = [[starts[0], None]]
    for i in range(len(starts) - 1):
        if ends[i] < starts[i + 1]:
            ranges[-1][1] = ends[i]
            ranges.append([starts[i + 1], None])
    ranges[-1][1] = ends[-1]
    return [{
        'begin': range[0],
        'end': range[1],
        'phrase': input_text[range[0]:range[1] + 1]
     } for range in ranges]

def remove_duplicates(phrases):
    """Удалите фразы, которые появляются несколько раз."""
    i = 0
    while i < len(phrases):
        j = i + 1
        while j < len(phrases):
            if phrases[i]['phrase'] == phrases[j]['phrase']:
                phrases.remove(phrases[j])
            j += 1
        i += 1

    return phrases

def get_output_lists(df_row):
    """Возвращает кортеж с двумя списками по 10 фраз в каждом. Первый объединяет ключевые фразы, 
    которые накладываются друг на друга, чтобы создать более длинные фразы, что лучше всего подходит для
    выделения ключевых фраз в тексте, а второй - это просто ключевые фразы с самыми высокими баллами, 
    что лучше всего подходит для подведения итогов документа."""
    keyphrases = []
    for row in df_row.keywords:
        keyphrases.append({
            'begin': row.begin,
            'end': row.end,
            'phrase': row.result,
            'score': adjusted_score(row)
        })
    keyphrases = sorted(keyphrases, key=lambda x: x['score'], reverse=True)

    return (
        get_top_ranges(keyphrases[:20], df_row.text)[:10],
        remove_duplicates(keyphrases[:10])[:10]
    )

Преобразуйте входные данные примера для создания фрейма данных, хранящего идентифицированные ключевые фразы.

In [100]:
df = spark.createDataFrame(pd.DataFrame({'text': input_list}))
result = light_pipeline.transform(df).toPandas()

Для каждого примера создайте два файла JSON, содержащих выбор лучших ключевых фраз для документа. См. docstring get_output_lists две ячейки выше, чтобы узнать больше о двух созданных файлах JSON. Эти JSON-файлы используются непосредственно в публичном демонстрационном приложении для этой модели.

In [101]:
! mkdir -p $OUTPUT_FILE_PATH

for i in range(len(result)):
    top_ranges, top_summaries = get_output_lists(result.iloc[i])
    with open(f'{OUTPUT_FILE_PATH}/Example{i + 1}.json', 'w') as ranges_file:
        json.dump(top_ranges, ranges_file)
    with open(f'{OUTPUT_FILE_PATH}/Example{i + 1}_summaries.json', 'w') \
            as summaries_file:
        json.dump(top_summaries, summaries_file)

## 5. Visualize outputs

The raw pandas data frame containing the outputs

In [85]:
result

,text,document,sentences,tokens,keywords
0,Extracting keywords from texts has become a ch...,"[(document, 0, 896, Extracting keywords from t...","[(document, 0, 135, Extracting keywords from t...","[(token, 0, 9, Extracting, {'sentence': '0'}, ...","[(keyword, 0, 18, extracting keywords, {'sente..."
1,Iodine deficiency is a lack of the trace eleme...,"[(document, 0, 1119, Iodine deficiency is a la...","[(document, 0, 90, Iodine deficiency is a lack...","[(token, 0, 5, Iodine, {'sentence': '0'}, []),...","[(keyword, 0, 16, iodine deficiency, {'sentenc..."
2,The Prague Quadrennial of Performance Design a...,"[(document, 0, 1548, The Prague Quadrennial of...","[(document, 0, 287, The Prague Quadrennial of ...","[(token, 0, 2, The, {'sentence': '0'}, []), (t...","[(keyword, 4, 21, prague quadrennial, {'senten..."
3,Author Nathan Wiseman-Trowse explained that th...,"[(document, 0, 2358, Author Nathan Wiseman-Tro...","[(document, 0, 205, Author Nathan Wiseman-Trow...","[(token, 0, 5, Author, {'sentence': '0'}, []),...","[(keyword, 0, 12, author nathan, {'sentence': ..."
4,North Ingria was located in the Karelian Isthm...,"[(document, 0, 1679, North Ingria was located ...","[(document, 0, 83, North Ingria was located in...","[(token, 0, 4, North, {'sentence': '0'}, []), ...","[(keyword, 0, 11, north ingria, {'sentence': '..."
5,"Учитесь учиться! Мы вступаем в век, в котором ...","[(document, 0, 3425, Учитесь учиться! Мы вступ...","[(document, 0, 15, Учитесь учиться!, {'sentenc...","[(token, 0, 6, Учитесь, {'sentence': '0'}, [])...","[(keyword, 20, 29, вступаем в, {'sentence': '1..."


In [81]:
with open('outputs/Example6_summaries.json', 'r', encoding='utf-8') as f: #открыли файл с данными
    text_sum = json.load(f) #загнали все, что получилось в переменную

text_sum #вывели результат на экран

[{'begin': 1544, 'end': 1551, 'phrase': 'знаний и', 'score': 17.8511658567772},
 {'begin': 2549, 'end': 2556, 'phrase': 'знаний и', 'score': 17.8511658567772},
 {'begin': 3073, 'end': 3078, 'phrase': 'и то и', 'score': 17.811434810617737},
 {'begin': 29, 'end': 33, 'phrase': 'в век', 'score': 15.03738309174792},
 {'begin': 888, 'end': 892, 'phrase': 'в век', 'score': 15.03738309174792},
 {'begin': 1880,
  'end': 1895,
  'phrase': 'знаний и навыков',
  'score': 14.171712113699739},
 {'begin': 3023,
  'end': 3033,
  'phrase': 'а не просто',
  'score': 12.260118911125272}]

Список верхних ключевых фраз (с перекрывающимися объединенными ключевыми фразами) для последнего примера

#### почему остались повторы?

In [102]:
top_ranges

[{'begin': 29, 'end': 33, 'phrase': 'в век'},
 {'begin': 706, 'end': 710, 'phrase': 'в век'},
 {'begin': 801, 'end': 809, 'phrase': 'не просто'},
 {'begin': 888, 'end': 900, 'phrase': 'в век машин и'},
 {'begin': 1239, 'end': 1250, 'phrase': 'не сможешь и'},
 {'begin': 1537, 'end': 1551, 'phrase': 'просто знаний и'},
 {'begin': 1880, 'end': 1895, 'phrase': 'знаний и навыков'},
 {'begin': 2111, 'end': 2125, 'phrase': 'и не радоваться'},
 {'begin': 2549, 'end': 2564, 'phrase': 'знаний и навыков'},
 {'begin': 3023, 'end': 3033, 'phrase': 'а не просто'}]

Список лучших сводных ключевых фраз (с удаленными дубликатами) для последнего примера

#### как фраза 'и то и' могла набрать большую оценку?

In [103]:
top_summaries

[{'begin': 3073, 'end': 3078, 'phrase': 'и то и', 'score': 28.533519923176826},
 {'begin': 1544,
  'end': 1551,
  'phrase': 'знаний и',
  'score': 21.27200525476634},
 {'begin': 2549,
  'end': 2556,
  'phrase': 'знаний и',
  'score': 21.27200525476634},
 {'begin': 1880,
  'end': 1895,
  'phrase': 'знаний и навыков',
  'score': 19.19131032447267},
 {'begin': 1239,
  'end': 1250,
  'phrase': 'не сможешь и',
  'score': 17.74329543860378},
 {'begin': 2111,
  'end': 2125,
  'phrase': 'и не радоваться',
  'score': 17.393712971631714},
 {'begin': 29, 'end': 33, 'phrase': 'в век', 'score': 17.355679151169674}]